In [ ]:
'''
Overview

download latest acra data
data are separated into multiple files alphabetically
combine multiple files into one
clean and reorganise columns to make it user friendly as columns are in a messy order
drop columns that are mostly missing values, using a user defined threshold
write clean df to file with the last data update date
'''

In [ ]:
# imports
import pandas as pd
import numpy as np
import glob
import requests
import os
import re
from zipfile import ZipFile
from io import BytesIO
from pathlib import Path
from tqdm.notebook import tqdm_notebook

In [ ]:
# define acra data download url
# https://data.gov.sg/dataset/acra-information-on-corporate-entities
acra_download_url = 'https://data.gov.sg/dataset/21d477f2-6e1b-4232-82b3-59e804dc2f6a/download'
# define folder to extract acra data files
acra_folder = 'acra-information-on-corporate-entities'
# define metadata file name
metadata_filename = 'metadata-acra-information-on-corporate-entities.txt'

In [ ]:
# download data from url
acra_data_zip = requests.get(acra_download_url)
# extract files from zip into a folder
acra_data_zip = ZipFile(BytesIO(acra_data_zip.content))
acra_data_zip.extractall(acra_folder)

In [ ]:
# read metadata as one string
metadata = Path(os.path.join(acra_folder, metadata_filename)).read_text()
# define regex pattern to extract last updated date
last_updated_date_pattern = 'Last Updated: \'(\d{4}-\d{2}-\d{2})'
# get first instance of 'Last Updated' date
last_updated = re.findall(last_updated_date_pattern, metadata)[0]

In [ ]:
# get a list of all csv files
list_of_files = glob.glob(acra_folder+'\\*.csv')

# create empty dataframe to store data
full_df = pd.DataFrame()
# loop through each file and concat to df
for file in tqdm_notebook(list_of_files):
    # read csv
    df = pd.read_csv(file)
    # concat df to full_df
    full_df = pd.concat([full_df, df])

# replace 'na' with nan
full_df = full_df.replace('na', np.nan)

In [ ]:
# define date columns
date_columns = ['uen_issue_date', 'registration_incorporation_date', 'account_due_date', 'annual_return_date']
# convert datetime columns to date
full_df[date_columns] = full_df[date_columns].apply(pd.to_datetime)

In [ ]:
# define organised list of columns
clean_columns = [
    'uen',
    'uen_issue_date',
    'registration_incorporation_date',
    'issuance_agency_id',
    'entity_name',
    'entity_status_description',
    'entity_type_description',
    'business_constitution_description',
    'company_type_description',
    'primary_ssic_code',
    'primary_ssic_description',
    'primary_user_described_activity',
    'secondary_ssic_code',
    'secondary_ssic_description',
    'secondary_user_described_activity',
    'address_type',
    'block',
    'street_name',
    'building_name',
    'level_no',
    'unit_no',
    'postal_code',
    'other_address_line1',
    'other_address_line2',
    'account_due_date',
    'annual_return_date',
    'no_of_charges',
    'no_of_officers',
    'paf_constitution_description',
    'uen_of_audit_firm1',
    'name_of_audit_firm1',
    'uen_of_audit_firm2',
    'name_of_audit_firm2',
    'uen_of_audit_firm3',
    'name_of_audit_firm3',
    'uen_of_audit_firm4',
    'name_of_audit_firm4',
    'uen_of_audit_firm5',
    'name_of_audit_firm5',
    'former_entity_name1',
    'former_entity_name2',
    'former_entity_name3',
    'former_entity_name4',
    'former_entity_name5',
    'former_entity_name6',
    'former_entity_name7',
    'former_entity_name8',
    'former_entity_name9',
    'former_entity_name10',
    'former_entity_name11',
    'former_entity_name12',
    'former_entity_name13',
    'former_entity_name14',
    'former_entity_name15',
    'paid_up_capital1_currency',
    'paid_up_capital1_ordinary',
    'paid_up_capital1_others',
    'paid_up_capital1_preference',
    'paid_up_capital2_currency',
    'paid_up_capital2_ordinary',
    'paid_up_capital2_others',
    'paid_up_capital2_preference',
    'paid_up_capital3_currency',
    'paid_up_capital3_ordinary',
    'paid_up_capital3_others',
    'paid_up_capital3_preference',
    'paid_up_capital4_currency',
    'paid_up_capital4_ordinary',
    'paid_up_capital4_others',
    'paid_up_capital4_preference',
    'paid_up_capital5_currency',
    'paid_up_capital5_ordinary',
    'paid_up_capital5_others',
    'paid_up_capital5_preference',
    'paid_up_capital6_currency',
    'paid_up_capital6_ordinary',
    'paid_up_capital6_others',
    'paid_up_capital6_preference',
    'paid_up_capital7_currency',
    'paid_up_capital7_ordinary',
    'paid_up_capital7_others',
    'paid_up_capital7_preference',
    'paid_up_capital8_currency',
    'paid_up_capital8_ordinary',
    'paid_up_capital8_others',
    'paid_up_capital8_preference',
    'paid_up_capital9_currency',
    'paid_up_capital9_ordinary',
    'paid_up_capital9_others',
    'paid_up_capital9_preference',
    'paid_up_capital10_currency',
    'paid_up_capital10_ordinary',
    'paid_up_capital10_others',
    'paid_up_capital10_preference'
    ]

# reorganise columns
full_df = full_df[clean_columns]

In [ ]:
# create df to store columns and the count percentage of nulls
missing_values_perc = pd.DataFrame(full_df.isnull().sum() / len(full_df)).reset_index().rename(columns={'index':'Column', 0:'Percentage Missing Values'})


##### adjust threshold here #####

# define threshold for missing values to drop column
# e.g. if 0.9 of rows in a column are missing values, drop the column
missing_value_threshold = 0.9 

##### adjust threshold here #####


# get list of columns to drop
list_of_columns_to_drop = list(missing_values_perc['Column'].loc[(missing_values_perc['Percentage Missing Values'] >= missing_value_threshold)])

# drop columns
clean_df = full_df.drop(list_of_columns_to_drop, axis=1).reset_index(drop=True)
# print list of columns to drop
print(f'The {len(list_of_columns_to_drop)} columns below have been dropped as at least {missing_value_threshold * 100}% of the values in the column are missing.')
print('\n')
print(list_of_columns_to_drop)

In [ ]:
# display
display(clean_df)

In [ ]:
# write to file
clean_df.to_csv(acra_folder+'-cleaned-'+last_updated+'.csv', index=False)